In [1]:
# завантажити потрібні бібліотеки
#!pip install gdown
#!pip install pyspark

# встановити джаву якщо віддалений сервак
# sudo apt update
# sudo apt install default-jdk -y

In [2]:
# взяти публічний ключ з гугл диска
#!gdown https://drive.google.com/file/d/1sVLrWAj-yILDf4mptzq4cIiZeMf_s8wF -O CT_2_practice_rsa.pub

In [3]:
# перевірка чи все ок і чи він доступний
#!ls CT_2_practice_rsa.pub

In [4]:
# зміна доступу для файла (занадто доступний не приймається на вхід)
#!chmod 600 CT_2_practice_rsa.pub

In [5]:
# перевірка доступності порта
#!ping -c 4 147.185.40.27
#!nc -zv 147.185.40.27 20057

In [6]:
# прокид портів 
# (ганяти в терміналі а не тут, бо .ipynb файл не дозволяє паралельне виконання клітинок)

#!ssh -i CT_2_practice_rsa.pub -L 9870:localhost:9870  -L 9009:localhost:9000 -L 9866:localhost:9866  -L 9864:localhost:986 -L 8088:localhost:8088 -p 20057 user@147.185.40.27

In [7]:
# TODO: HW_4

# Домашнє завдання: Визначення оптимальної ціни квитка з використанням розширеного аналізу даних

# Об'єднання кількох джерел даних:

# З'єднати таблицю фільмів (movies_metadata), наприклад, з таблицею рейтингів (ratings) Done
# Інтегрувати дані про акторів (credits) для аналізу впливу зірок на ціноутворення
# Додати дані про прокат фільмів (сезонність, тривалість прокату)

# Створення Silver Layer:

# Нормалізувати JSON-структури (genres, cast, keywords)
# Створити додаткові метрики (ROI, прибуток на день прокату, відношення бюджет/рейтинг) Done
# Обробити аномалії та відсутні дані з використанням stateful операцій

In [8]:
## Опис виконаного завдання 

# Загальна ідея передбачення - [Фіча-інжиніринг] -> [Кластеризація] -> [FFNN] -> [Ціна]

### Фіча - інжиніринг
# Для реалістичних результатів вирішено було поєднати дані про фільми, жанри та акторів (3/3)

# На основі нових даних було утворено нові фічі: 
# - вплив акторів на успіх фільму 
# (середній та максимальний ROI, середня та максимальна кількість фільмів де взято участь серед усіх акторів на фільм) (Done)
# 
# - конкурентне оточення (кількість фільмів того ж жанру в попередні 2 тижні від релізу) 
# 
# - конкурентне оточення по року (відхилення ROI фільму від ROI усіх фільмів упродовж того ж року) (Done) 
# (для конкуретного оточення використано own_ROI/avg(ROI))
# 
# - конкурентне оточення по жанру (відхилення ROI фільму від ROI усіх фільмів із того ж жанру) (Done)


### Кластеризація
# Ідея - групувати фільми на кілька кластерів за схожістю характеристик 
# 
# Дозволить створити базові цінові кошики для кожного кластеру

### FFNN 
# Ідея - використати алгоритм, потужніший за звичайну лінійну регресію 
# (це дозволить виявити і працильно відреагувати на деякі ситуації, наприклад порогові ефекти та мультиплікативні взаємодії) 
# 
# При цьому поєднання мережі з кластеризацією дозволить забезпечити належність ціну до певного цінового кошика
# 
# Приблизна архітектура: 
# Input (13 features + cluster_id) 
# -> Dense(128) -> BatchNorm -> Dropout 
# -> Dense(64) → Tanh 
# -> Dense(32) -> L2-regularization 
# -> Output(price)

In [9]:
# імпорти

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, year, to_date, avg, min, max, abs, count, expr, desc, explode, array_contains, when, datediff, current_date, monotonically_increasing_id, lit
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, ArrayType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from functools import reduce
import json
import pandas as pd
import numpy as np
import os
import io
import matplotlib.pyplot as plt

In [10]:
# 0.0 Spark session establish

# for CPU
#"""
spark = SparkSession.builder \
	.master("local").appName("hdfs_test")\
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9009") \
    .getOrCreate()
#"""

# for GPU
"""
spark = SparkSession.builder \
    .master("local") \
    .appName("hdfs_test") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9009") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.task.cpus", "4") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.rapids.memory.gpu.pool.size", "2G") \
    .getOrCreate()
"""

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 06:54:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'\nspark = SparkSession.builder     .master("local")     .appName("hdfs_test")     .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9009")     .config("spark.plugins", "com.nvidia.spark.SQLPlugin")     .config("spark.executor.memory", "8g")     .config("spark.executor.cores", "4")     .config("spark.task.cpus", "4")     .config("spark.rapids.sql.enabled", "true")     .config("spark.sql.execution.arrow.enabled", "true")     .config("spark.rapids.memory.gpu.pool.size", "2G")     .getOrCreate()\n'

In [11]:
# 0.1 Завантаження даних


import os

hdfs_path = "hdfs://localhost:9009/user/hadoop/imported-data/2025-03-04/"
silver_layer_path = "file:///" + os.path.join(os.getcwd(), "silver_layer")
results_path = "file:///" + os.path.join(os.getcwd(), "results")


movies_df = spark.read.csv(
    f"{hdfs_path}movies.csv", 
    header=True, 
    inferSchema=True
    )


movies_metadata_df = spark.read.csv(
    f"{hdfs_path}movies_metadata.csv", 
    header=True, 
    inferSchema=True
    )


ratings_df = spark.read.csv(
    f"{hdfs_path}matsevytyi_table2_rankings.csv", 
    header=True, 
    inferSchema=True
    )


actors_df = spark.read.csv(
    f"{hdfs_path}actors_sychova.csv", 
    header=True, 
    inferSchema=True
    )

In [12]:
## 1. ОБ'ЄДНАННЯ ДАНИХ ТА ПОБУДОВА SILVER LAYER

In [13]:
# 1.1 Підготовка movies_df до Silver Layer

# Обираємо ключові колонки для подальшого аналізу
# Конвертуємо значення полів до оптимальних
# Додаємо нові поля
# Додаємо stateful операцію для пошуку відхилення значень від середнього упродовж певногго періоду

print("Оптимізація таблиці...")

movies_optimized = movies_df.select(
    
    # Це не міняється
    col("id").cast("integer").alias("movie_id"),
    col("title"),
    
    # Парсинг року з date випуску для партиціонування
    col("release_date"),
    year(to_date(col("release_date"))).alias("release_year"),
    
    # Явне приведення типів і відповідне перейменування
    col("vote_average").cast("double").alias("vote_average_numeric"),
    col("popularity").cast("double").alias("popularity_numeric"),
    col("budget").cast("double").alias("budget_numeric"),
    col("revenue").cast("double").alias("revenue_numeric"),
    
    # Збереження JSON полів для подальшої обробки
    col("genres")
    
)

print("Очищення даних...")

movies_cleaned = movies_optimized.filter(
    
    # Фільтрація відсутніх та нульових значень бюджету
    # Нульові бюджети можуть вказувати на неповні дані і спотворювати аналіз
    
    (col("budget_numeric").isNotNull()) & 
    (col("budget_numeric") > 0) &
    
    # Фільми без зазначеного доходу не можуть бути використані для аналізу рентабельності
    
    (col("revenue_numeric").isNotNull()) & 
    (col("revenue_numeric") > 0) &
    
    # Фільми без рейтингу не можуть бути використані для аналізу популярності
    
    (col("vote_average_numeric").isNotNull()) &
    (col("vote_average_numeric") > 0)
    
)

movies_cleaned = movies_cleaned.dropDuplicates(["title"])

print("Обчислення додаткових метрик...")

movies_with_metrics = movies_cleaned \
    .withColumn( 
    "ROI", # ROI (коефіцієнт рентабельності інвестицій) є ключовим показником фінансового успіху фільму
    col("revenue_numeric") / col("budget_numeric")
    ) \
    .withColumn(
    "profit_per_day", # прибуток за день
    col("revenue_numeric") / datediff(current_date(), col("release_date"))
    ) \
    .withColumn(
    "ROI_rating_ratio", # відношення виручка/рейтинг допомогає оцінити наскільки вдало була підібрана ціна квитка
    col("ROI") / col("vote_average_numeric") 
    )

# Валідація схеми

expected_columns = ["movie_id", "title", "release_date", "budget_numeric", "revenue_numeric"]
assert all(c in movies_with_metrics.columns for c in expected_columns)

print(f"...Having {movies_with_metrics.count()} lines of records overall")
movies_with_metrics.show()

Оптимізація таблиці...
Очищення даних...
Обчислення додаткових метрик...


...Having 9600 lines of records overall


+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+
|movie_id|               title|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|                ROI|     profit_per_day|    ROI_rating_ratio|
+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+
|  516517|"Beyond Wrestling...|  2018-04-06|        2018|                 9.0|             1.407|        9888.0|         7764.0|        Action-Drama| 0.7851941747572816|  3.017489312087058|  0.0872437971952535|
|  466287|"How Viktor ""The...|  2018-03-23|        2018|                 7.1|             2.698|      441755.0|        95589.0|        Drama-Comedy|0.21638

In [14]:
# Показ загальної статистики для Silver Layer
print("\n=== ЗАГАЛЬНА СТАТИСТИКА ФІЛЬМІВ ===")

movies_stats = movies_with_metrics.select(
    count("*").alias("total_count"),
    avg("budget_numeric").alias("avg_budget"),
    avg("revenue_numeric").alias("avg_revenue"),
    avg("vote_average_numeric").alias("avg_rating"),
    avg("roi").alias("avg_roi")
).collect()[0]

print(f"Загальна кількість фільмів після очищення: {movies_stats['total_count']}")
print(f"Середній бюджет: ${movies_stats['avg_budget']:,.2f}")
print(f"Середній прибуток: ${movies_stats['avg_revenue']:,.2f}")
print(f"Середній рейтинг: {movies_stats['avg_rating']:.2f}")
print(f"Середній ROI: {movies_stats['avg_roi']:.2f}x")


=== ЗАГАЛЬНА СТАТИСТИКА ФІЛЬМІВ ===


Загальна кількість фільмів після очищення: 9600
Середній бюджет: $24,242,237.82
Середній прибуток: $68,500,970.41
Середній рейтинг: 6.54
Середній ROI: 21112.51x


In [15]:
# Об'єднання кількох джерел даних

print("...Об'єднуємо movies_metadata, movies та rantings")

common_titles = movies_metadata_df \
    .select("title") \
    .intersect(movies_with_metrics.select("title"))
        

joined_movies = common_titles.join(movies_with_metrics, on="title", how="inner")

joined_movies = joined_movies.join(movies_metadata_df, on="title", how="inner")

average_ranks = ratings_df.groupBy("movieId").agg(avg("rating").alias("average_rating"))

joined_with_ranks = joined_movies.join(average_ranks, on=joined_movies.movie_id==average_ranks.movieId, how="left")

print(f"...Having {joined_with_ranks.count()} lines of records overall")

joined_with_ranks.show()

...Об'єднуємо movies_metadata, movies та rantings


...Having 7779 lines of records overall


25/04/22 06:56:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+------------------+------------------+--------------------+-----+---------------------+---------+--------------------+--------------------+------+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+-----+------------+----------+-------+------------------+
|               title|movie_id|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|               ROI|    profit_per_day|    ROI_rating_ratio|adult|belongs_to_collection|   budget|              genres|            homepage|    id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|pro

In [16]:
# 1.2 Genres to Silver

# Нормалізуємо залежність movies <-> genres до 3 рівня нормалізації
# Треба окрема таблиця для genres і заміна з назви на 


def parse_genres(genres_json):
    """
    
    Функція для парсингу JSON полів жанрів
    Жанри в датасеті зберігаються у форматі JSON, потрібно їх перетворити в зручний формат
    
    """
    
    try:
        if genres_json and genres_json.strip():
            # Заміна одинарних лапок на подвійні для коректного парсингу JSON
            genres = json.loads(genres_json.replace("'", "\""))
            # Вилучення назв жанрів зі структурованого JSON
            return [genre.get('name', '') for genre in genres if 'name' in genre]
        return []
    except:
        return []

def split_genres(genres_str):
    if genres_str and genres_str.strip():
        return [g.strip() for g in genres_str.split("-")]
    return []


if 'genres' in movies_df.columns:
    
    print("\n=== АНАЛІЗ ЖАНРІВ ===")
    
    # Реєстрація UDF функції для використання в Spark SQL щоб застосовувати Python-функції до колонок DataFrame
    parse_genres_udf = spark.udf.register("parse_genres", parse_genres, ArrayType(StringType()))
    split_genres_udf = spark.udf.register("split_genres", split_genres, ArrayType(StringType()))
    
    # Обчислення жанрів
    movies_with_genres = movies_with_metrics.withColumn("genre_list", split_genres_udf(col("genres")))

    movies_with_genres.show(10)
    
    # Вибух (explode) масиву жанрів для аналізу
    genres_exploded = movies_with_genres.select(
        "movie_id", "title", explode(col("genre_list")).alias("genre")
    )
    
    # Підрахунок фільмів за жанрами
    # Визначення найпопулярніших жанрів за кількістю фільмів
    genre_counts = genres_exploded.groupBy("genre").count().orderBy(desc("count"))
    print("Найпопулярніші жанри:")
    genre_counts.show(10)
    
    # Показ середньої рентабельності за жанрами
    # Дозволяє визначити, які жанри найбільш прибуткові
    print("\nРентабельність за жанрами:")
    genres_roi = movies_with_genres.select(
        "title", explode(col("genre_list")).alias("genre"), 
        col("budget_numeric"), col("revenue_numeric"), col("roi")
    ).filter(
        # Фільтрація для включення лише фільмів з коректними фінансовими даними
        (col("budget_numeric") > 0) & 
        (col("revenue_numeric") > 0) & 
        col("roi").isNotNull()
    ).groupBy("genre").agg(
        # Обчислення середнього ROI для кожного жанру
        avg("roi").alias("average_roi"),
        # Підрахунок кількості фільмів для кожного жанру для оцінки статистичної значущості
        count("title").alias("movie_count")
        # Фільтрація жанрів з малою кількістю фільмів для підвищення статистичної значущості
    ).filter(col("movie_count") > 10).orderBy(desc("average_roi"))
    
    genres_roi.show(10)
    
    # Окрема таблиця для жанрів
    
    genres_roi = genres_roi.withColumn("genre_id", monotonically_increasing_id())
    
    # Окрема таблиця для many-to-many зв'язку між movies та genres
    
    movie_genre_relation = genres_exploded.join(
        genres_roi, genres_exploded["genre"] == genres_roi["genre"], "inner"
    ).select(
        genres_roi["genre_id"], genres_exploded["movie_id"].alias("film_id")
    )
    


=== АНАЛІЗ ЖАНРІВ ===


+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+--------------------+
|movie_id|               title|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|                ROI|     profit_per_day|    ROI_rating_ratio|          genre_list|
+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+--------------------+
|  516517|"Beyond Wrestling...|  2018-04-06|        2018|                 9.0|             1.407|        9888.0|         7764.0|        Action-Drama| 0.7851941747572816|  3.017489312087058|  0.0872437971952535|     [Action, Drama]|
|  466287|"How Viktor ""The...|  2018-03-23|        2018|               

+---------------+-----+
|          genre|count|
+---------------+-----+
|          Drama| 4476|
|         Comedy| 3397|
|         Action| 2375|
|       Thriller| 2226|
|        Romance| 1750|
|      Adventure| 1531|
|          Crime| 1389|
|         Horror| 1065|
|Science Fiction|  942|
|         Family|  918|
+---------------+-----+
only showing top 10 rows


Рентабельність за жанрами:


+-----------+------------------+-----------+
|      genre|       average_roi|movie_count|
+-----------+------------------+-----------+
|     Horror| 94855.14997248157|       1065|
|   TV Movie| 49215.98103159122|         21|
|    Mystery|1302.9860563666039|        774|
|     Comedy|1183.8108191182107|       3397|
|      Drama| 629.6503820668494|       4476|
|    Romance| 620.3787291653363|       1750|
|     Family|154.58040832077367|        918|
|Documentary| 49.30017380135712|        192|
|  Adventure|15.809311218959344|       1531|
|     Action| 10.67295903929723|       2375|
+-----------+------------------+-----------+
only showing top 10 rows



In [17]:
# Зберегти таблиці в дельту
    
#!mkdir silver_layer
#!mkdir silver_layer/genres
#!mkdir silver_layer/genres/genres
#!mkdir silver_layer/genres/movie_genre_relation

genres_roi.show(10)
movie_genre_relation.show(10)

genres_roi.write.mode("overwrite").parquet(f"{silver_layer_path}/genres/genres")
movie_genre_relation.write.mode("overwrite").parquet(f"{silver_layer_path}/genres/movie_genre_relation")

+-----------+------------------+-----------+--------+
|      genre|       average_roi|movie_count|genre_id|
+-----------+------------------+-----------+--------+
|     Horror| 94855.14997248157|       1065|       0|
|   TV Movie| 49215.98103159122|         21|       1|
|    Mystery|1302.9860563666039|        774|       2|
|     Comedy|1183.8108191182107|       3397|       3|
|      Drama| 629.6503820668494|       4476|       4|
|    Romance| 620.3787291653363|       1750|       5|
|     Family|154.58040832077367|        918|       6|
|Documentary| 49.30017380135712|        192|       7|
|  Adventure|15.809311218959344|       1531|       8|
|     Action| 10.67295903929723|       2375|       9|
+-----------+------------------+-----------+--------+
only showing top 10 rows



+--------+-------+
|genre_id|film_id|
+--------+-------+
|       0| 932428|
|       0| 381283|
|       0| 427396|
|       0|   1946|
|       0| 185341|
|       0|  80280|
|       0|  10664|
|       0|   8329|
|       0| 338967|
|       0|  19908|
+--------+-------+
only showing top 10 rows



In [18]:
## Оновлення movies_optimized


#movies_expoded = m

# Нові колонки до фільму (відхиленя ROI та популярності від середньої за роком та жанром)

movies_exploded = movies_with_genres.withColumn("genre", explode(col("genre_list")))

# Потім join з genres_roi
movies_with_genres_roi = movies_exploded.join(
    genres_roi, 
    on="genre", 
    how="left"
)

movies_with_genres_roi.show(10)


# Обчислюємо стандартне відхилення для кожного фільма за його жанрами
movies_with_roi_deviation = movies_with_genres_roi.withColumn(
    "roi_deviation", col("ROI") / col("average_roi")
)

movies_with_roi_deviation.show(10)

# Для кожного фільма обчислюємо середнє значення відхилень від всіх його жанрів
movies_with_roi_deviation_avg = movies_with_roi_deviation.groupBy("movie_id").agg(
    avg("roi_deviation").alias("roi_deviation_by_genres")
)

movies_with_roi_deviation_avg.show(10)

# Приєднуємо це значення назад до оригінальних фільмів
movies_with_metrics = movies_with_metrics.join(
    movies_with_roi_deviation_avg, "movie_id", "left"
)


# Групування по року випуску
year_window = Window.partitionBy("release_year")

# Додавання відхилення від середнього значення за рік (щоб зрозуміти і локальну успішність та очікуваність фільму)
movies_with_metrics = movies_with_metrics \
    .withColumn("avg_roi", avg("ROI").over(year_window)) \
    .withColumn("roi_deviation_over_year", col("ROI") / col("avg_roi"))  
    

# Підчищення дублікатів
movies_with_metrics = movies_with_metrics.drop("genre_list").dropDuplicates()

# Перевірка результату
movies_with_metrics.show()

# DONE

+--------+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+-----------+--------+
|   genre|movie_id|               title|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|                ROI|     profit_per_day|    ROI_rating_ratio|          genre_list|       average_roi|movie_count|genre_id|
+--------+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+-----------+--------+
|Thriller|  614696|              #Alive|  2020-06-24|        2020|                 7.3|            42.048|     6300000.0|    1.3416285E7|Action-Horror-Thr...

+--------+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+-----------+--------+--------------------+
|   genre|movie_id|               title|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|                ROI|     profit_per_day|    ROI_rating_ratio|          genre_list|       average_roi|movie_count|genre_id|       roi_deviation|
+--------+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+-----------+--------+--------------------+
|Thriller|  614696|              #Alive|  2020-06-24|        2020|                 7.3|       

+--------+-----------------------+
|movie_id|roi_deviation_by_genres|
+--------+-----------------------+
|    1645|     0.3700387356693662|
|    2142|    0.20028598799060662|
|   10623|    0.15131763774935017|
|    8592|    0.12486679575846975|
|   11033|    0.35679683378438354|
|    1580|    0.14257245978263208|
|  382614|    0.04468387764171299|
|  468284|   0.017891071043987537|
|   50348|    0.20757148235834758|
|    2122|    0.06340205616570933|
+--------+-----------------------+
only showing top 10 rows



+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+------------------+-------------------+-------------------+-----------------------+------------------+-----------------------+
|movie_id|               title|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|               ROI|     profit_per_day|   ROI_rating_ratio|roi_deviation_by_genres|           avg_roi|roi_deviation_over_year|
+--------+--------------------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+------------------+-------------------+-------------------+-----------------------+------------------+-----------------------+
|  402388|The Girl of the G...|  1915-01-04|        1915|                 5.4|               1.4|       15110.0|       102224.0|     Western-Romance| 6.765320979483786| 2.5

In [19]:
# 1.3 Actors to Silver Layer
# Окрема таблиця для акторів з додатковими обчисленими фічами
# Окрема таблиця для взаємозалежності кіно-актор

actors_with_movies = actors_df.select(
    col("movie_title").alias("title"),
    col("actor_name")
)

actors_with_movies = actors_with_movies.filter(
    col("title").isNotNull() & \
    col("actor_name").isNotNull()
)

actors_with_movies = actors_with_movies.withColumn(
    "relation_id", 
    monotonically_increasing_id()
    )

actors_roi = actors_with_movies.join(
    movies_with_metrics, "title",
).select(
    actors_with_movies.actor_name,
    "title", 
    "ROI"
)

actors_roi.show(10)

actors_final = actors_roi.groupBy("actor_name").agg(
    count("title").alias("movies_count"),
    avg("ROI").alias("avg_ROI")
)
print(f"... actos_final and _with_ of {actors_final.count()} and {actors_with_movies.count()} length")
actors_final.show(10)
actors_with_movies.show(10)

# Зберігаємо результат
actors_final.write.mode("overwrite").parquet(f"{silver_layer_path}/actors/actors")
actors_with_movies.write.mode("overwrite").parquet(f"{silver_layer_path}/actors/actors_with_movies")

+------------------+--------------------+------------+
|        actor_name|               title|         ROI|
+------------------+--------------------+------------+
|     Bhanwar Singh|The Darjeeling Li...|1.5235719375|
|       Owen Wilson|The Darjeeling Li...|1.5235719375|
|      Adrien Brody|The Darjeeling Li...|1.5235719375|
| Jason Schwartzman|The Darjeeling Li...|1.5235719375|
|       Amara Karan|The Darjeeling Li...|1.5235719375|
|Wallace Wolodarsky|The Darjeeling Li...|1.5235719375|
|   Waris Ahluwalia|The Darjeeling Li...|1.5235719375|
|       Irrfan Khan|The Darjeeling Li...|1.5235719375|
|  Barbet Schroeder|The Darjeeling Li...|1.5235719375|
|Camilla Rutherford|The Darjeeling Li...|1.5235719375|
+------------------+--------------------+------------+
only showing top 10 rows



... actos_final and _with_ of 237165 and 5798446 length


+----------------+------------+------------------+
|      actor_name|movies_count|           avg_ROI|
+----------------+------------+------------------+
|Jhalaram Bishnoi|           1|      1.5235719375|
|      Snoop Dogg|          23| 3.026047542581171|
|    Edmund Ikeda|           3| 2.529612964268349|
|       Ava Acres|          14| 4.872347474957527|
|   Lucille Bliss|           7| 26.88446004910557|
|Bradford Dillman|          11|5.2374858660255965|
|   Locklyn Black|           1| 4.676378857142857|
|Daniel Day-Lewis|          16|3.5701683327008964|
|    KPAC Lalitha|          19| 5.532554577210519|
| Viktor Horinyak|           4|3.6691971104601917|
+----------------+------------+------------------+
only showing top 10 rows

+------+-----------------+-----------+
| title|       actor_name|relation_id|
+------+-----------------+-----------+
|Barbie|     Ryan Gosling|          0|
|Barbie|  America Ferrera|          1|
|Barbie|Ariana Greenblatt|          2|
|Barbie|         Issa Rae|

In [20]:
## Нові дані для movies_with_metrics

# join full actors metrics with movie
joined = actors_with_movies.join(
    actors_final,
    on="actor_name",  # спільна колонка
    how="left"        # залишаємо всі записи з фільмів, навіть якщо немає метрик
)

joined.show(10)

# Calculate avg and max ROI for each movie of its actors
movie_actor_avg_max_roi = joined.groupBy("title") \
.agg(
    avg("avg_ROI").alias("avg_actor_ROI"),
    max("avg_ROI").alias("max_actor_ROI")
)

# Aggregate these counts per movie
movie_actor_avg_max_movies = joined.groupBy("title") \
.agg(
    avg("movies_count").alias("avg_movies_per_actor"),
    max("movies_count").alias("max_movies_per_actor")
)

# Join the movie ROI and movie actor statistics
actor_movie_stats = movie_actor_avg_max_roi.join(
    movie_actor_avg_max_movies, "title", "inner"
)

actor_movie_stats.show(10)

movies_with_metrics = movies_with_metrics.join (
    actor_movie_stats, "title", "inner" # щоб уникнути фільмів без записів про акторів
)

# Це фінальні значення, зберегти
movies_with_metrics.write.mode("overwrite").parquet(f"{silver_layer_path}/movies/movies_with_metrics")

# Показати
print(f"...Having {movies_with_metrics.count()} lines of records overall")
movies_with_metrics.show()

+-----------------+------------------+-----------+------------+------------------+
|       actor_name|             title|relation_id|movies_count|           avg_ROI|
+-----------------+------------------+-----------+------------+------------------+
|  Alexandra Shipp|            Barbie|          5|           9| 4.558178566799342|
|  America Ferrera|            Barbie|          1|          11|3.3386055905285703|
|   Ana Cruz Kayne|            Barbie|          9|           3|11.590926831609195|
|Ariana Greenblatt|            Barbie|          2|           9|  2.86555354364503|
|Bahare Kianafshar|        Golshifteh| 8589934597|           3|1.4882935575399934|
| Chicco Kurniawan|Kenapa Harus Bule?| 8589934594|        NULL|              NULL|
|       Dina Cocea|      Boga Sisters|17179869194|        NULL|              NULL|
| Djenar Maesa Ayu|Kenapa Harus Bule?| 8589934592|        NULL|              NULL|
|    Doina Tutescu|      Boga Sisters|17179869193|        NULL|              NULL|
|   

+-------------------+------------------+-----------------+--------------------+--------------------+
|              title|     avg_actor_ROI|    max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|
+-------------------+------------------+-----------------+--------------------+--------------------+
|            !Boomer|              NULL|             NULL|                NULL|                NULL|
|                 #1|              NULL|             NULL|                NULL|                NULL|
|#1 Cheerleader Camp|2.8830922995646406|8.600333657265326|  2.5454545454545454|                   9|
|         #132642667|              NULL|             NULL|                NULL|                NULL|
|         #1915House|      4.7511369375|     4.7511369375|                 1.0|                   1|
|                 #2|              NULL|             NULL|                NULL|                NULL|
|            #21xoxo|              42.0|             42.0|                 1.0|            

...Having 8829 lines of records overall


+--------------------+--------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+-----------------------+------------------+------------------+--------------------+--------------------+
|               title|movie_id|release_date|release_year|vote_average_numeric|popularity_numeric|budget_numeric|revenue_numeric|              genres|                 ROI|      profit_per_day|    ROI_rating_ratio|roi_deviation_by_genres|           avg_roi|roi_deviation_over_year|     avg_actor_ROI|     max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|
+--------------------+--------+------------+------------+--------------------+------------------+--------------+---------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------+-----------------------+-

In [21]:
# Clen-up a bit

del joined, movie_actor_avg_max_roi, movie_actor_avg_max_movies, actor_movie_stats, actors_with_movies, actors_roi, actors_final
del movies_with_roi_deviation, movies_with_roi_deviation_avg, genres_exploded, movie_genre_relation, genres_roi
del movies_df, movies_metadata_df, ratings_df, actors_df
del movies_cleaned, movies_optimized
del common_titles, joined_movies, average_ranks, joined_with_ranks


import gc
gc.collect()

319

In [22]:
## МОДЕЛЮВАННЯ

In [23]:
## 2.1 Кластеризація фільмів

# Кластеризація фільмів для аналізу
print("\n=== КЛАСТЕРНИЙ АНАЛІЗ ФІЛЬМІВ ===")

# Підготовка даних для кластеризації

movies_with_metrics.printSchema()


# Вибір числових ознак для кластеризації
# Використання ключових метрик фільмів для групування
features = ["release_year",
            "budget_numeric", "revenue_numeric", "vote_average_numeric" , "popularity_numeric",
            "ROI", "profit_per_day", "ROI_rating_ratio",
            "roi_deviation_by_genres",  "roi_deviation_over_year",
            "avg_actor_ROI", "max_actor_ROI", "avg_movies_per_actor", "max_movies_per_actor"]

# Фільтрація для видалення відсутніх значень
# Кластеризація вимагає повних даних без пропусків
filered_movies_with_metrics = movies_with_metrics.select("movie_id", "title", *features).na.drop()

print("...Векторизація даних")
    
# Створення вектора ознак для алгоритму кластеризації
# VectorAssembler об'єднує кілька колонок в один вектор ознак
assembler = VectorAssembler(inputCols=features, outputCol="features")
vectorized_data = assembler.transform(filered_movies_with_metrics)

print("...Нормалізація даних")
# Нормалізація ознак для урівноваження їх впливу на кластеризацію
# StandardScaler приводить всі ознаки до однакового масштабу
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(vectorized_data)
scaled_data = scaler_model.transform(vectorized_data)

print("...Кластеризація даних")
# Кластеризація K-Means з 5 кластерами
# K-Means групує фільми у 5 природних кластерів на основі їх характеристик
kmeans = KMeans(k=5, featuresCol="scaled_features")
kmeans_model = kmeans.fit(scaled_data)

print("...Оновлення початкових даних з присвоєнням кластерів")
# Додавання інформації про кластери до даних
# Кожному фільму присвоюється номер кластера
clustered_data = kmeans_model.transform(scaled_data)

print("...Аналіз кластерів")
# Аналіз кластерів
cluster_summary = clustered_data.groupBy("prediction").agg(
    count("*").alias("count"),
    *[avg(f).alias("AVG_" + f) for f in features]
).orderBy("prediction")


print("Характеристики кластерів:")
cluster_summary.show()

# Зразки фільмів з кожного кластера
print("\nПриклади фільмів з кожного кластера:")
for cluster_id in range(5):
    print(f"\nКластер {cluster_id}:")
    
    clustered_data \
        .filter(col("prediction") == cluster_id) \
        .show(3)





=== КЛАСТЕРНИЙ АНАЛІЗ ФІЛЬМІВ ===
root
 |-- title: string (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- release_date: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- vote_average_numeric: double (nullable = true)
 |-- popularity_numeric: double (nullable = true)
 |-- budget_numeric: double (nullable = true)
 |-- revenue_numeric: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- ROI: double (nullable = true)
 |-- profit_per_day: double (nullable = true)
 |-- ROI_rating_ratio: double (nullable = true)
 |-- roi_deviation_by_genres: double (nullable = true)
 |-- avg_roi: double (nullable = true)
 |-- roi_deviation_over_year: double (nullable = true)
 |-- avg_actor_ROI: double (nullable = true)
 |-- max_actor_ROI: double (nullable = true)
 |-- avg_movies_per_actor: double (nullable = true)
 |-- max_movies_per_actor: long (nullable = true)

...Векторизація даних


...Нормалізація даних


...Кластеризація даних


25/04/22 07:14:58 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


...Оновлення початкових даних з присвоєнням кластерів
...Аналіз кластерів
Характеристики кластерів:


...

+--------+--------------------+------------+--------------+---------------+--------------------+------------------+-------------------+------------------+-------------------+-----------------------+-----------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|               title|release_year|budget_numeric|revenue_numeric|vote_average_numeric|popularity_numeric|                ROI|    profit_per_day|   ROI_rating_ratio|roi_deviation_by_genres|roi_deviation_over_year|     avg_actor_ROI|     max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|            features|     scaled_features|prediction|
+--------+--------------------+------------+--------------+---------------+--------------------+------------------+-------------------+------------------+-------------------+-----------------------+-----------------------+------------------+------------------+--------------------+-------

+--------+--------------------+------------+--------------+---------------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|               title|release_year|budget_numeric|revenue_numeric|vote_average_numeric|popularity_numeric|               ROI|    profit_per_day|  ROI_rating_ratio|roi_deviation_by_genres|roi_deviation_over_year|     avg_actor_ROI|     max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|            features|     scaled_features|prediction|
+--------+--------------------+------------+--------------+---------------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+------------------+------------------+--------------------+-------------

+--------+----------------+------------+--------------+---------------+--------------------+------------------+------------------+------------------+-------------------+-----------------------+-----------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|           title|release_year|budget_numeric|revenue_numeric|vote_average_numeric|popularity_numeric|               ROI|    profit_per_day|   ROI_rating_ratio|roi_deviation_by_genres|roi_deviation_over_year|     avg_actor_ROI|    max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|            features|     scaled_features|prediction|
+--------+----------------+------------+--------------+---------------+--------------------+------------------+------------------+------------------+-------------------+-----------------------+-----------------------+------------------+-----------------+--------------------+--------------------+----

+--------+--------------------+------------+--------------+---------------+--------------------+------------------+-----------------+------------------+------------------+-----------------------+-----------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|               title|release_year|budget_numeric|revenue_numeric|vote_average_numeric|popularity_numeric|              ROI|    profit_per_day|  ROI_rating_ratio|roi_deviation_by_genres|roi_deviation_over_year|     avg_actor_ROI|     max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|            features|     scaled_features|prediction|
+--------+--------------------+------------+--------------+---------------+--------------------+------------------+-----------------+------------------+------------------+-----------------------+-----------------------+------------------+------------------+--------------------+----------------

+--------+--------------------+------------+--------------+---------------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|movie_id|               title|release_year|budget_numeric|revenue_numeric|vote_average_numeric|popularity_numeric|               ROI|    profit_per_day|  ROI_rating_ratio|roi_deviation_by_genres|roi_deviation_over_year|     avg_actor_ROI|     max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|            features|     scaled_features|prediction|
+--------+--------------------+------------+--------------+---------------+--------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------------+------------------+------------------+--------------------+-------------

In [24]:
# Зберегти дані кластеру

clustered_data.write.mode("overwrite").parquet(f"{results_path}/clusters/movie_price_clusters")

# Завантажити дані кластеру

#clustered_data = spark.read.parquet(f"{results_path}/clusters/movie_price_clusters")

In [25]:
# Формування рекомендацій на основі попередніх оцінок (якщо є дані)
print("\n=== ПРИКЛАД РЕКОМЕНДАЦІЙ ===")


# Створення рекомендацій на основі найпопулярніших фільмів
popular_movies = movies_with_metrics.filter(
    (col("vote_average_numeric") > 7.5) & (col("popularity_numeric") > 50)
).orderBy(desc("popularity_numeric")).limit(10)

print("Рекомендовані фільми на основі популярності:")
popular_movies.select("title", "vote_average_numeric", "popularity_numeric").show()

# Приклад персоналізованих рекомендацій (імітація)
#print("\nПриклад персоналізованих рекомендацій для користувача:")

# Імітація уподобань користувача (наприклад, вподобання для жанру "Action")
if 'genre_list' in locals():
    action_movies = movies_with_genres.filter(
        array_contains(col("genre_list"), "Action")
    ).filter(
        col("vote_average_numeric") > 7.0
    ).orderBy(desc("vote_average_numeric")).limit(5)
    
    print("Рекомендації для любителів бойовиків:")
    action_movies.select("title", "vote_average_numeric").show()



=== ПРИКЛАД РЕКОМЕНДАЦІЙ ===
Рекомендовані фільми на основі популярності:


+--------------------+--------------------+------------------+
|               title|vote_average_numeric|popularity_numeric|
+--------------------+--------------------+------------------+
|Deadpool & Wolverine|               7.765|          5410.496|
|      Dune: Part Two|                 8.3|          4742.163|
|Spider-Man: Acros...|                8.64|          2550.738|
|Furiosa: A Mad Ma...|               7.652|          1872.179|
|Puss in Boots: Th...|               8.284|            904.25|
|           Migration|               7.563|           800.053|
|John Wick: Chapter 4|               7.827|           659.169|
|Guardians of the ...|               7.623|            459.47|
|Killers of the Fl...|               7.525|           452.285|
|           Star Wars|                 8.2|           441.814|
+--------------------+--------------------+------------------+



In [26]:
# Подивитися межі по середніх відхиленнях (щоб зрозуміти чи треба їх локально нормалізувати)

# Середнє відхилення не розглядаємо бо кластеризація відбувалася на тих же фічах що і FFNN модель, вілповідно буде трохи overhead

from pyspark.sql.functions import lit, round as spark_round, sqrt, log, least, greatest

# ROI deviation by genres — min and max
movies_with_metrics.select("roi_deviation_by_genres").summary("min", "max").show()

# ROI deviation over year — min and max
movies_with_metrics.select("roi_deviation_over_year").summary("min", "max").show()


# Оцінка загальної статистики щоб правильно оптимізувати
clustered_data.select("roi_deviation_by_genres").summary().show()
clustered_data.groupBy(spark_round("roi_deviation_by_genres", 1).alias("raw_rounded")).count().orderBy("raw_rounded").show()

clustered_data.select("roi_deviation_over_year").summary().show()
clustered_data.groupBy(spark_round("roi_deviation_over_year", 1).alias("raw_rounded")).count().orderBy("raw_rounded").show()

print("All clustered data size: ", clustered_data.count())
non_null_roi_deviation_by_genres = clustered_data.na.drop(subset=["roi_deviation_by_genres"])
non_null_roi_deviation_over_year = clustered_data.na.drop(subset=["roi_deviation_over_year"])
print("Valid roi_deviation_by_genres clustered data size: ", non_null_roi_deviation_by_genres.count())
print("Valid roi_deviation_over_year clustered data size: ", non_null_roi_deviation_over_year.count())

valid_cluster = clustered_data.filter((col("roi_deviation_by_genres") <= 0) | (col("roi_deviation_over_year") <= 0))
print(valid_cluster.count())
valid_cluster.show()

+-------+-----------------------+
|summary|roi_deviation_by_genres|
+-------+-----------------------+
|    min|   3.103037161513903...|
|    max|     2455.2195204207455|
+-------+-----------------------+



+-------+-----------------------+
|summary|roi_deviation_over_year|
+-------+-----------------------+
|    min|   3.001072668025444...|
|    max|      298.2896075410536|
+-------+-----------------------+



+-------+-----------------------+
|summary|roi_deviation_by_genres|
+-------+-----------------------+
|  count|                   8751|
|   mean|     1.2289157031966622|
| stddev|      41.25709218301797|
|    min|   3.103037161513903...|
|    25%|   0.003645483458104...|
|    50%|    0.05043665031446451|
|    75%|     0.2129726621598099|
|    max|     2455.2195204207455|
+-------+-----------------------+



+-----------+-----+
|raw_rounded|count|
+-----------+-----+
|        0.0| 4369|
|        0.1| 1599|
|        0.2|  863|
|        0.3|  597|
|        0.4|  345|
|        0.5|  197|
|        0.6|  128|
|        0.7|  106|
|        0.8|   83|
|        0.9|   51|
|        1.0|   52|
|        1.1|   29|
|        1.2|   34|
|        1.3|   25|
|        1.4|   25|
|        1.5|   19|
|        1.6|   18|
|        1.7|    9|
|        1.8|   11|
|        1.9|    9|
+-----------+-----+
only showing top 20 rows



+-------+-----------------------+
|summary|roi_deviation_over_year|
+-------+-----------------------+
|  count|                   8751|
|   mean|     0.9501489365059208|
| stddev|      7.599292703647563|
|    min|   1.716103408545812...|
|    25%|    0.03176332121417424|
|    50%|    0.20288698970451985|
|    75%|     0.6270605021111284|
|    max|     298.28960754105356|
+-------+-----------------------+



...

All clustered data size:  8751


Valid roi_deviation_by_genres clustered data size:  8751


Valid roi_deviation_over_year clustered data size:  8751


0


+--------+-----+------------+--------------+---------------+--------------------+------------------+---+--------------+----------------+-----------------------+-----------------------+-------------+-------------+--------------------+--------------------+--------+---------------+----------+
|movie_id|title|release_year|budget_numeric|revenue_numeric|vote_average_numeric|popularity_numeric|ROI|profit_per_day|ROI_rating_ratio|roi_deviation_by_genres|roi_deviation_over_year|avg_actor_ROI|max_actor_ROI|avg_movies_per_actor|max_movies_per_actor|features|scaled_features|prediction|
+--------+-----+------------+--------------+---------------+--------------------+------------------+---+--------------+----------------+-----------------------+-----------------------+-------------+-------------+--------------------+--------------------+--------+---------------+----------+
+--------+-----+------------+--------------+---------------+--------------------+------------------+---+--------------+--------

In [27]:
# Аналіз для визначення оптимальної ціни квитка
print("\n=== ВИЗНАЧЕННЯ ОПТИМАЛЬНОЇ ЦІНИ КВИТКА ===")

# Оскільки немає тренувальних цін для ціни квитка, вирішено використати weak supervision -
# тобто побудувати проксі-ціну, на її основі натренувати модель і встановити ці ціни як нові

# Проксі: empirically_predicted_price = 5.0 < 10.0 * roi_deviation_by_genres * roi_deviation_over_year < 20.0

# оскільки деякі фільми мають roi_deviation > 3, що свідчить не стільки про високу ціну квитків, скільки про захопленість фільмами, 
# це треба стабілізувати, для цього використаємо clip+log+1 (стягуємо deviations до одиниці і обрізаємро мж 5 і 20)

# Орієнтовна архітектура моделі
# Input (14 features + cluster_id) 
# -> Dense(128) -> BatchNorm -> Dropout 
# -> Dense(64) → Tanh 
# -> Dense(32) -> L2-regularization 
# -> Output(price)


# Імпортуємо необхідні функції
from pyspark.sql.functions import lit, round as spark_round, sqrt, log, least, greatest
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Приналежність до конкретного кластера також впливає а ціну квитка
clustered_data = clustered_data.withColumnRenamed("prediction", "cluster_id")

# Додати константу щоб уникнути логарифму з від'ємного числа
shift_roi_deviation_by_genres = clustered_data.select(min("roi_deviation_by_genres")).first()[0] + 0.0001 #зміщення щоб не було логарифму від нуля
shift_roi_deviation_by_year = clustered_data.select(min("roi_deviation_over_year")).first()[0] + 0.0001 #зміщення щоб не було логарифму від нуля

print("...Calculating raw_pricing_data")
# Розрахуємо проксі-ціну квитка
# Використовуємо формулу, яка враховує бюджет, рейтинг і популярність
raw_pricing_data = clustered_data.withColumn(

    "proxy_ticket_price", 

    # стягування ROI до одиниці
    10.0 * ( 
        (log(col("roi_deviation_by_genres") + shift_roi_deviation_by_genres) + 1 ) * 
        (log(col("roi_deviation_over_year") + shift_roi_deviation_by_year) + 1 )
        )
    )  

print("...Refining raw_pricing_data")
# Потім обрізаємо результат між 5 і 20
# І округлимо ціну до 2 знаків після коми
pricing_data = raw_pricing_data.withColumn(
    "proxy_ticket_price", 
    spark_round(
            least(
                    greatest(
                        col("proxy_ticket_price")
                    , lit(5.0))
            , lit(20.0))
    , 2)
)


print("All pricing data size: ", pricing_data.count())
non_null_pricing_data = pricing_data.na.drop(subset=["proxy_ticket_price"])
print("Valid pricing data size: ", non_null_pricing_data.count())

# Покажемо розподіл цін
print("\nРозподіл базових цін на квитки:")
price_distribution = pricing_data.groupBy(
    spark_round(col("proxy_ticket_price"), 0).alias("price_rounded")
).count().orderBy("price_rounded")

price_distribution.show(20)

# Знайдемо середню ціну квитка
avg_price = pricing_data.select(avg("proxy_ticket_price")).collect()[0][0]
print(f"\nСередня базова ціна квитка: ${avg_price:.2f}")

# Аналіз цін по жанрах
if 'genre_list' in locals():
    print("\nСередня ціна квитка за жанрами:")
    genre_pricing = pricing_data.withColumn(
        "genre_list", parse_genres_udf(col("genres"))
    ).select(
        "title", explode(col("genre_list")).alias("genre"), "proxy_ticket_price"
    ).groupBy("genre").agg(
        avg("proxy_ticket_price").alias("avg_price"),
        count("title").alias("movie_count")
    ).filter(col("movie_count") > 10).orderBy(desc("avg_price"))
    
    genre_pricing.show(15)



=== ВИЗНАЧЕННЯ ОПТИМАЛЬНОЇ ЦІНИ КВИТКА ===


25/04/22 07:48:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 07:48:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 07:48:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 07:48:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 07:48:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 07:48:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


...Calculating raw_pricing_data
...Refining raw_pricing_data


All pricing data size:  8751


Valid pricing data size:  8751

Розподіл базових цін на квитки:


+-------------+-----+
|price_rounded|count|
+-------------+-----+
|          5.0| 3768|
|          6.0|  126|
|          7.0|  118|
|          8.0|   99|
|          9.0|   94|
|         10.0|   86|
|         11.0|  102|
|         12.0|   72|
|         13.0|   90|
|         14.0|   83|
|         15.0|   72|
|         16.0|   73|
|         17.0|   74|
|         18.0|   50|
|         19.0|   49|
|         20.0| 3795|
+-------------+-----+




Середня базова ціна квитка: $12.40


In [28]:
# Save/load pricing_data

In [51]:
## МОДЕЛЬ І ТРЕНУВАННЯ

#!pip install elephas 
# (installs tf too)

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GeneralizedLinearRegression
from elephas.ml_model import ElephasEstimator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, serialize

if not "cluster_id" in pricing_data.columns:
    print("cluster_id not exists, do clustering first")

# 1. Підготовка features vector
if "features" in pricing_data.columns:

    pricing_data = pricing_data.drop("features")

print("...Converting features")

features = ["release_year",
            "budget_numeric", "revenue_numeric", "vote_average_numeric" , "popularity_numeric",
            "ROI", "profit_per_day", "ROI_rating_ratio",
            "roi_deviation_by_genres",  "roi_deviation_over_year",
            "avg_actor_ROI", "max_actor_ROI", "avg_movies_per_actor", "max_movies_per_actor",
            "cluster_id"]

assembler = VectorAssembler(
    inputCols=features,
    outputCol="features"
)

assembled_data = assembler.transform(pricing_data)#.select("features", "proxy_ticket_price")

# 2. Побудова моделі
print("...Setting up model architecture")

optimizer_config = Adam(learning_rate=0.001)

model = Sequential()
model.add(Dense(128, input_shape=(15,), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1))  # регресія в кінці

model.compile(loss='mse', optimizer=optimizer_config, metrics=['mae'])

# 3. Обгортка в Elephas Estimator для роботи зі Спарком
print("...Converting to Elephas Estimator")


estimator = ElephasEstimator()
estimator._set(
    keras_model_config=model.to_json(),
    optimizer_config={"class_name": "Adam", "config": {"learning_rate": 0.001}},  # важно
    loss='mse',
    metrics=['mae'],
    epochs=50,
    batch_size=32,
    featuresCol="features",
    labelCol="proxy_ticket_price"
)

# 4. Навчання
print("...Training the model")
print(f"...estimating {len(features)}x1 input dimension")

# switch fpr this because having some interna; issue with Elephas
fitted_model = GeneralizedLinearRegression(featuresCol="features", labelCol="proxy_ticket_price", family="gaussian", link="identity", 
                                          regParam=0.1).fit(assembled_data)

print("...fitted successfully")


...Converting features
...Setting up model architecture
...Converting to Elephas Estimator
...Training the model
...estimating 15x1 input dimension


25/04/22 09:01:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
25/04/22 09:01:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 09:01:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 09:01:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/04/22 09:01:58 WARN RowBasedKeyValueBatch: Calling spill() on Ro

...fitted successfully


In [54]:
from pyspark.sql.functions import abs as spark_abs

assembled_data = assembled_data.drop("features")

features = ["release_year",
            "budget_numeric", "revenue_numeric", "vote_average_numeric" , "popularity_numeric",
            "ROI", "profit_per_day", "ROI_rating_ratio",
            "roi_deviation_by_genres",  "roi_deviation_over_year",
            "avg_actor_ROI", "max_actor_ROI", "avg_movies_per_actor", "max_movies_per_actor",
            "cluster_id"]

assembler = VectorAssembler(
    inputCols=features,
    outputCol="features"
)

assembled_data = assembler.transform(assembled_data)#.select("features", "proxy_ticket_price")


#assembled_data.printSchema()

if "ideal_price" in assembled_data.columns:
    assembled_data = assembled_datacing_data.drop("ideal_price")


if "raw_ideal_price" in pricing_data.columns:
    assembled_data = assembled_data.drop("raw_ideal_price")

print("...Calculating ideal_price with our trained model")
# Розрахуємо ідеальну ціну для усього датафрейму
post_model_data = fitted_model.transform(assembled_data)
post_model_data = post_model_data.withColumnRenamed("prediction", "raw_ideal_price")

print("...small processing of ideal_price")

if "ideal_price" in post_model_data.columns:
    post_model_data = post_model_data.drop("ideal_price")

# Округлимо до 2 знаків
post_model_data = post_model_data.withColumn(
    "ideal_price", 
    spark_round(15 + spark_abs(col("raw_ideal_price")), 2)
)

post_model_data = post_model_data.withColumn(
    "ideal_price",
    when(col("ideal_price") < 5, 5)
    .when(col("ideal_price") > 30, 30)
    .otherwise(col("ideal_price"))
)

# І потім використовуємо її в коді:
post_model_data.select(
    "title", 
    "proxy_ticket_price", 
    spark_round(col("ideal_price"), 2).alias("ideal_price"), 
    spark_round(col("proxy_ticket_price") - col("ideal_price"), 2).alias("difference")
).orderBy(spark_abs(col("difference")).desc()).show(10)

...Calculating ideal_price with our trained model
...small processing of ideal_price


+--------------------+------------------+-----------+----------+
|               title|proxy_ticket_price|ideal_price|difference|
+--------------------+------------------+-----------+----------+
|              Cuckoo|               5.0|       30.0|     -25.0|
| Come Forth Reverend|               5.0|       30.0|     -25.0|
|           Crossover|               5.0|       30.0|     -25.0|
|    A Stroke of Luck|               5.0|       30.0|     -25.0|
|           Dandelion|               5.0|       30.0|     -25.0|
|Bañeros 4: Los ro...|               5.0|       30.0|     -25.0|
|           Action 3D|               5.0|       30.0|     -25.0|
|America: Imagine ...|               5.0|       30.0|     -25.0|
|          Dark World|               5.0|       30.0|     -25.0|
|   Bharya Athra Pora|               5.0|       30.0|     -25.0|
+--------------------+------------------+-----------+----------+
only showing top 10 rows

